载入特征文件（）并建立网络，分类，进行训练，

In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(0)

In [2]:
train_features = []
test_features = []
train_labels = []
train_files = []

train_files_temp = []
with h5py.File("modelfile/xception_bottleneck.h5", 'r') as h:
    train_features.append(np.array(h['train_features']))
    test_features.append(np.array(h['test_features']))
    train_labels.append(np.array(h['train_label']))
    train_files_temp.append(np.array(h['train_file_que']))

In [3]:
train_features = train_features[0]
test_features = test_features[0]
train_labels = train_labels[0]
for file in train_files_temp[0]:
    train_files.append(file.decode())

In [4]:
#验证数据集的形状是否正确
print("train_features shape:",np.shape(train_features))
print("test_features shape:",np.shape(test_features))
print("train_labels shape:",np.shape(train_labels))
print("train_features data type:",train_features.dtype)

train_features shape: (60, 2048)
test_features shape: (10, 2048)
train_labels shape: (60,)
train_features data type: float32


In [5]:
#是否打乱
train_features, train_labels,train_files = shuffle(train_features, train_labels,train_files)

构建网络

In [2]:
from keras.layers import Input, Dropout, Dense
from keras.models import Model
from keras import optimizers
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [7]:
inputlayer = Input(shape=(train_features.shape[1:]))

In [8]:
x = inputlayer

x = Dropout(0.8)(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.8)(x)
outlayer = Dense(1, kernel_initializer="glorot_uniform", activation='sigmoid')(x)

model = Model(inputs=inputlayer, outputs=outlayer)

In [9]:
#优化器参考：https://keras.io/optimizers/
# Stochastic gradient descent optimizer.
opt = optimizers.SGD(lr=0.005, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_features,train_labels,epochs=30,verbose=1,validation_split=0.2, batch_size=128, callbacks=[TensorBoard(log_dir='./log')])
model.save('modelfile/model.h5')

Train on 48 samples, validate on 12 samples
Epoch 1/20
48/48 [==============================] - 0s 805us/step - loss: 0.9470 - acc: 0.4792 - val_loss: 0.7074 - val_acc: 0.5000
Epoch 2/20
48/48 [==============================] - 0s 432us/step - loss: 0.9424 - acc: 0.5208 - val_loss: 0.7001 - val_acc: 0.5000
Epoch 3/20
48/48 [==============================] - 0s 498us/step - loss: 0.9883 - acc: 0.5833 - val_loss: 0.6903 - val_acc: 0.5000
Epoch 4/20
48/48 [==============================] - 0s 549us/step - loss: 0.8778 - acc: 0.4583 - val_loss: 0.6782 - val_acc: 0.5833
Epoch 5/20
48/48 [==============================] - 0s 481us/step - loss: 0.8567 - acc: 0.5208 - val_loss: 0.6636 - val_acc: 0.5833
Epoch 6/20
48/48 [==============================] - 0s 404us/step - loss: 0.7602 - acc: 0.6458 - val_loss: 0.6473 - val_acc: 0.5833
Epoch 7/20
48/48 [==============================] - 0s 463us/step - loss: 0.9576 - acc: 0.4167 - val_loss: 0.6300 - val_acc: 0.7500
Epoch 8/20
48/48 [==============

In [10]:
# 都预测为0.5的情况下，loss = 0.69314
pred = model.predict(train_features, batch_size=None, verbose=0, steps=None)

预测并打印对比真实标签和预测值

In [11]:
label = list(map(lambda x: "%.3f"%float(x), train_labels[:10]))
pred_p = list(map(lambda x: "%.3f"%float(x), pred.flatten()[:10]))
print("  label:",label)
print("predict:",pred_p)

  label: ['0.000', '1.000', '1.000', '0.000', '0.000', '0.000', '1.000', '1.000', '1.000', '0.000']
predict: ['0.393', '0.676', '0.610', '0.267', '0.413', '0.266', '0.709', '0.700', '0.752', '0.286']


此处可以打印10张图片，验证是否正确